# Extensions to Linear Models - Lab

## Introduction

In this lab, you'll practice many concepts learned in this section, from adding interactions and polynomials to your model to AIC and BIC!

## Summary

You will be able to:
- Build a linear regression model with polynomial features/interactions
- Perform regularization
- Use AIC and BIC to select the best value for the regularization parameter


## Let's get started!

Import all the necessary packages.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import preprocessing
from itertools import combinations

from sklearn.datasets import load_boston

## Look at a Baseline Boston Housing Data Model

Import the Boston housing data set, use all the predictors in their scaled version (using `preprocessing.scale`. Look at a baseline model using *scaled variables* as predictors. Use 5-fold cross-validation this time and use the $R^2$ score to evaluate the model.

In [2]:
bos = load_boston()

In [3]:
df = pd.DataFrame(preprocessing.scale(bos.data))
df.columns = bos.feature_names

In [4]:
bos.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

In [5]:
df.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')

In [6]:
y=bos.target

In [7]:
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,-0.419782,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562
1,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439
2,-0.417342,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727
3,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517
4,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501


## Include interactions

Look at all the possible combinations of variables for interactions by adding interactions one by one to the baseline model. Next, evaluate that model using 5-fold classification and store the $R^2$ to compare it with the baseline model.

You've created code for this before in the interactions lab, yet this time, you have scaled the variables so the outcomes may look different. 

Print the 7 most important interactions.

In [8]:
combos = []
for comb1, comb2 in combinations(df.columns, 2):
    model = LinearRegression()
    df2 = df
    df2["interaction"] = df[comb1]*df[comb2]
    model.fit(df2, y)
    combos.append([comb1, comb2, model.score(df2,y)])
for comb in sorted(combos, key=lambda x: x[2], reverse=True):
    print(comb)

['RM', 'LSTAT', 0.804710962672808]
['RM', 'TAX', 0.7943408370486366]
['RM', 'RAD', 0.7897010816912517]
['RM', 'PTRATIO', 0.7818647263865117]
['INDUS', 'RM', 0.7787951855353925]
['NOX', 'RM', 0.7678416665069798]
['RM', 'AGE', 0.7644048831798201]
['CRIM', 'RM', 0.7614772707404173]
['RM', 'B', 0.7602386028435114]
['RM', 'DIS', 0.7591046539312967]
['INDUS', 'DIS', 0.7538752751404021]
['ZN', 'RM', 0.7537746218880714]
['CRIM', 'CHAS', 0.7512052891664784]
['ZN', 'INDUS', 0.7465404025884856]
['DIS', 'TAX', 0.7461784004312446]
['INDUS', 'AGE', 0.7460362665575673]
['CRIM', 'DIS', 0.7455292683324883]
['CHAS', 'TAX', 0.7451627494084894]
['AGE', 'DIS', 0.7450697115083921]
['CHAS', 'RAD', 0.7448760291537604]
['NOX', 'DIS', 0.7448072231163838]
['INDUS', 'TAX', 0.7444070084073413]
['CRIM', 'NOX', 0.7443909146148453]
['DIS', 'RAD', 0.744347965355325]
['B', 'LSTAT', 0.7442008137462226]
['CHAS', 'LSTAT', 0.743825591433624]
['CRIM', 'RAD', 0.7438254523689352]
['ZN', 'LSTAT', 0.7437701840980503]
['NOX', 'R

Write code to include the 7 most important interactions in your data set by adding 7 columns. Name the columns "var1_var2" with var1 and var2 the two variables in the interaction.

In [9]:
for comb in sorted(combos, key=lambda x: x[2], reverse=True)[0:6]:
    df["%s_%s" % (comb[0], comb[1])] = df[comb[0]] * df[comb[1]]

## Include Polynomials

Try polynomials of 2, 3 and 4 for each variable, in a similar way you did for interactions (by looking at your baseline model and seeing how $R^2$ increases). Do understand that when going for a polynomial of 4, the particular column is raised to the power of 2 and 3 as well in other terms. We only want to include "pure" polynomials, so make sure no interactions are included. We want the result to return a list that contain tuples of the form:

`(var_name, degree, R2)`, so eg. `('DIS', 3, 0.732)`

In [10]:
from sklearn.preprocessing import PolynomialFeatures
import statsmodels.api as sm

In [11]:
for col in df.columns:
    df[col] = df[col].fillna(value=df[col].median())

In [12]:
df = df.replace([np.inf, -np.inf], np.nan)

In [13]:
results = []
for scale in [2,3,4]:
    for feature in df.columns:
        poly = PolynomialFeatures(scale)
        feature_poly = pd.DataFrame(poly.fit_transform(sm.add_constant(df[feature])))
        X_poly = pd.concat([df[[col for col in df.columns if col != feature]], feature_poly], axis=1)
        reg_poly = LinearRegression().fit(X_poly, y)
        results.append([feature, scale, reg_poly.score(X_poly,y)])

for result in sorted(results, key=lambda x: x[2], reverse=True):
    print(result)


['INDUS_RM', 4, 0.8388053790125998]
['RM_LSTAT', 4, 0.8381132110988259]
['DIS', 4, 0.8373855619224188]
['RM', 4, 0.8363912675600744]
['DIS', 3, 0.8361273552567948]
['DIS', 2, 0.8332976554458746]
['RM_TAX', 4, 0.8327528172495917]
['NOX_RM', 4, 0.8323293941474971]
['LSTAT', 4, 0.8322095447609035]
['RM_RAD', 4, 0.8313005886724807]
['TAX', 4, 0.830717304456799]
['RM_TAX', 3, 0.8305130064492949]
['RM_RAD', 3, 0.8301901461446913]
['NOX_RM', 3, 0.8298947201594007]
['RM_LSTAT', 3, 0.829537678026821]
['NOX', 4, 0.8294056788784564]
['TAX', 3, 0.8293235696311251]
['RM', 3, 0.8290306379182395]
['LSTAT', 3, 0.8290106727039587]
['INDUS_RM', 3, 0.8289796213988189]
['RM', 2, 0.8289743969320962]
['CRIM', 4, 0.8283959426143311]
['RM_PTRATIO', 4, 0.8281082013275358]
['INDUS_RM', 2, 0.8280391467020635]
['CRIM', 3, 0.8278084470644337]
['CRIM', 2, 0.827799061409417]
['AGE', 4, 0.8275611316889205]
['AGE', 3, 0.8275341682968153]
['LSTAT', 2, 0.8273855234866025]
['B', 4, 0.8273659301873806]
['INDUS', 4, 0.8273

For each variable, print out the maximum R2 possible when including Polynomials.

In [14]:
# Your code here

Which two variables seem to benefit most from adding Polynomial terms?


In [14]:
#['INDUS_RM', 4, 0.8388053790125998]
#['RM_LSTAT', 4, 0.8381132110988259]



Add Polynomials for the two features that seem to benefit the most, as in have the best R squared compared to the baseline model. For each of the two feature, raise to the Polynomial that generates the best result. Make sure to start from the data set `df_inter` so the final data set has both interactions and polynomials in the model.

In [15]:
df["INDUS_RM_4"] = df["INDUS_RM"]**4
df["RM_LSTAT_4"] = df["RM_LSTAT"]**4

In [16]:
df2 = df.drop(columns=["INDUS_RM","RM_LSTAT" ])

check out your final data set and make sure that your interaction terms as well as your polynomial terms are included.

In [17]:
y = pd.DataFrame(y)

In [18]:
cols = list(df2.columns)
cols[-1] = "price"
df2.columns = cols

## Full model R-squared

Check out the R-squared of the full model.

In [19]:
X = df2[[col for col in df2.columns if col !="price"]]

In [20]:
from sklearn.linear_model import LinearRegression

In [21]:
df2

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,interaction,RM_TAX,RM_RAD,RM_PTRATIO,NOX_RM,INDUS_RM_4,price
0,-0.419782,0.284830,-1.287909,-0.272599,-0.144217,0.413672,-0.120013,0.140214,-0.982843,-0.666608,-1.459000,0.441052,-1.075562,-0.474379,-0.275757,-0.406574,-0.603547,-0.059659,8.056853e-02,3.918919e-02
1,-0.417339,-0.487722,-0.593381,-0.272599,-0.740262,0.194274,0.367166,0.557160,-0.867883,-0.987329,-0.303094,0.441052,-0.492439,-0.217191,-0.191813,-0.168607,-0.058883,-0.143814,1.766027e-04,8.376717e-05
2,-0.417342,-0.487722,-0.593381,-0.272599,-0.740262,1.282714,-0.265812,0.557160,-0.867883,-0.987329,-0.303094,0.396427,-1.208727,-0.479172,-1.266461,-1.113245,-0.388783,-0.949544,3.356244e-01,5.778730e+00
3,-0.416750,-0.487722,-1.306878,-0.272599,-0.835284,1.016303,-0.809889,1.077737,-0.752922,-1.106115,0.113032,0.416163,-1.361517,-0.566613,-1.124148,-0.765197,0.114875,-0.848901,3.111944e+00,3.665929e+00
4,-0.412482,-0.487722,-1.306878,-0.272599,-0.835284,1.228577,-0.511180,1.077737,-0.752922,-1.106115,0.113032,0.441052,-1.026501,-0.452740,-1.358947,-0.925023,0.138869,-1.026210,6.645824e+00,2.529574e+00
5,-0.417044,-0.487722,-1.306878,-0.272599,-0.835284,0.207096,-0.351157,1.077737,-0.752922,-1.106115,0.113032,0.410571,-1.043322,-0.428358,-0.229072,-0.155927,0.023409,-0.172984,5.365743e-03,2.179536e-03
6,-0.410243,0.048772,-0.476654,-0.272599,-0.265154,-0.388411,-0.070229,0.839244,-0.523001,-0.577519,-1.505237,0.426798,-0.031268,-0.013345,0.224315,0.203139,0.584650,0.102989,1.174836e-03,2.175430e-08
7,-0.403696,0.048772,-0.476654,-0.272599,-0.265154,-0.160466,0.978808,1.024638,-0.523001,-0.577519,-1.505237,0.441052,0.910700,0.401666,0.092672,0.083924,0.241539,0.042548,3.422462e-05,4.560674e-04
8,-0.395935,0.048772,-0.476654,-0.272599,-0.265154,-0.931206,1.117494,1.087196,-0.523001,-0.577519,-1.505237,0.328448,2.421774,0.795427,0.537789,0.487022,1.401685,0.246913,3.881451e-02,2.586527e+01
9,-0.400729,0.048772,-0.476654,-0.272599,-0.265154,-0.399808,0.616090,1.329635,-0.523001,-0.577519,-1.505237,0.329325,0.623344,0.205283,0.230897,0.209100,0.601806,0.106011,1.318919e-03,3.857606e-03


In [22]:
model = LinearRegression()
model.fit(df2, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [56]:
model.score(df2,y)

0.815093960290724

In [66]:
all = pd.concat([df2,y], axis=1)

In [67]:

all.columns = list(df2.columns) + ["price"]

In [68]:
formula = "price~" + "+".join(df2.columns)

In [69]:
all.tail()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,...,B,LSTAT,interaction,RM_TAX,RM_RAD,RM_PTRATIO,NOX_RM,INDUS_RM_4,RM_LSTAT_4,price
501,-0.413229,-0.487722,0.115738,-0.272599,0.158124,0.439316,0.018673,-0.625796,-0.982843,-0.803212,...,0.387217,-0.418147,-0.161914,-0.352864,-0.431778,0.516840,0.069466,6.683702e-06,0.001139,22.4
502,-0.415249,-0.487722,0.115738,-0.272599,0.158124,-0.234548,0.288933,-0.716639,-0.982843,-0.803212,...,0.441052,-0.500850,-0.220901,0.188392,0.230524,-0.275937,-0.037088,5.430446e-07,0.000190,20.6
503,-0.413447,-0.487722,0.115738,-0.272599,0.158124,0.984960,0.797449,-0.773684,-0.982843,-0.803212,...,0.441052,-0.983048,-0.433575,-0.791131,-0.968061,1.158772,0.155746,1.688824e-04,0.878967,23.9
504,-0.407764,-0.487722,0.115738,-0.272599,0.158124,0.725672,0.736996,-0.668437,-0.982843,-0.803212,...,0.403225,-0.865302,-0.348911,-0.582868,-0.713222,0.853728,0.114746,4.975902e-05,0.155465,22.0
505,-0.415000,-0.487722,0.115738,-0.272599,0.158124,-0.362767,0.434732,-0.613246,-0.982843,-0.803212,...,0.441052,-0.669058,-0.295089,0.291379,0.356543,-0.426783,-0.057362,3.107574e-06,0.003470,11.9


In [70]:

model = ols(formula=formula, data=all).fit()

In [71]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.815
Model:                            OLS   Adj. R-squared:                  0.807
Method:                 Least Squares   F-statistic:                     106.9
Date:                Thu, 13 Jun 2019   Prob (F-statistic):          2.64e-163
Time:                        18:28:01   Log-Likelihood:                -1413.2
No. Observations:                 506   AIC:                             2868.
Df Residuals:                     485   BIC:                             2957.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      21.2706      0.220     96.892      0.000      20.839      21.702
CRIM           -1.1974      0.243     -4.920      0.000      -1.676      -0.719
ZN             -0.0649      0.291     -0.223      0.824      -0.636       0.507
INDUS           0.6558      0.365      1.797      0.073      -0.061       1.373
CHAS            0.7612      0.188      4.041      0.000       0.391       1.131
NOX            -1.8364      0.403     -4.558      0.000      -2.628      -1.045
RM              2.6970      0.291      9.261      0.000       2.125       3.269
AGE             0.0709      0.328      0.216      0.829      -0.573       0.715
DIS            -1.7961      0.371     -4.840      0.000      -2.525      -1.067
RAD             2.4927      0.521      4.780      0.000       1.468       3.517
TAX            -2.2310      0.550     -4.053      0.000      -3.313      -1.149
PTRATIO        -1.4654      0.246     -5.949      0.000      -1.949      -0.981
B               0.9996      0.303      3.304      0.001       0.405       1.594
LSTAT          -4.0390      0.335    -12.071      0.000      -4.697      -3.382
interaction    -0.3945      0.221     -1.783      0.075      -0.829       0.040
RM_TAX         -0.3497      0.519     -0.673      0.501      -1.370       0.671
RM_RAD          0.1018      0.520      0.196      0.845      -0.920       1.124
RM_PTRATIO     -1.3372      0.262     -5.105      0.000      -1.852      -0.823
NOX_RM         -1.2808      0.300     -4.269      0.000      -1.870      -0.691
INDUS_RM_4      0.0184      0.009      2.153      0.032       0.002       0.035
RM_LSTAT_4      0.0007      0.000      2.200      0.028    7.02e-05       0.001
==============================================================================
Omnibus:                      271.141   Durbin-Watson:                   0.970
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2519.176
Skew:                           2.158   Prob(JB):                         0.00
Kurtosis:                      13.042   Cond. No.                     3.01e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.01e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Finding the best Lasso regularization parameter

You've learned that, when using Lasso regularization, your coefficients shrink to 0 when using a higher regularization parameter. Now the question is which value we should choose for the regularization parameter. 

This is where the AIC and BIC come in handy! We'll use both criteria in what follows and perform cross-validation to select an optimal value of the regularization parameter alpha of the Lasso estimator.

Read the page here: https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_model_selection.html and create a similar plot as the first one listed on the page. 

In [10]:
# Your code here

## Analyze the final result

Finally, use the best value for regularization parameter according to AIC and BIC and compare the R squared parameters and MSE using train-test-split. Compare with the baseline model.

In [11]:
# Code for baseline model

In [12]:
# code for lasso with alpha from AIC

In [13]:
# code for lasso with alpha from BIC

## Level Up - Optional

### Create a Lasso Path

From this section, you know that when using lasso, more parameters shrink to zero as your regularization parameter goes up. In Scikit-Learn there is a function lasso_path which visualizes the shrinkage of the coefficients while alpha changes. Try this out yourself!

https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_coordinate_descent_path.html#sphx-glr-auto-examples-linear-model-plot-lasso-coordinate-descent-path-py

### AIC and BIC for subset selection
This notebook shows how you can use AIC and BIC purely for feature selection. Try this code out on our Boston Housing data!

https://xavierbourretsicotte.github.io/subset_selection.html

## Summary

Congratulations! You now know how to create better linear models and how to use AIC and BIC for both feature selection and to optimize your regularization parameter when performing Ridge and Lasso. 